In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [6]:
import sys, os
import torch

sys.path.append(os.path.abspath('../'))


from src.task_vectors_adapted import TaskVectorAdapted
from src.eval import eval_single_dataset
from src.args import parse_arguments 

In [7]:
from src.datasets import registry

In [8]:
registry.registry

{'CIFAR10': src.datasets.cifar10.CIFAR10,
 'CIFAR100': src.datasets.cifar100.CIFAR100,
 'Cars': src.datasets.cars.Cars,
 'DTD': src.datasets.dtd.DTD,
 'EuroSAT': src.datasets.eurosat.EuroSAT,
 'EuroSATVal': src.datasets.eurosat.EuroSATVal,
 'GTSRB': src.datasets.gtsrb.GTSRB,
 'ImageNet': src.datasets.imagenet.ImageNet,
 'MNIST': src.datasets.mnist.MNIST,
 'RESISC45': src.datasets.resisc45.RESISC45,
 'STL10': src.datasets.stl10.STL10,
 'SUN397': src.datasets.sun397.SUN397,
 'SVHN': src.datasets.svhn.SVHN}

In [9]:
#Config
datasets = ['Cars', 'MNIST']
model = 'ViT-L-14'

sys.argv = [
    'notebook',  # script name
    '--data-location', '../datasets',
    '--model', f'../{model}',
    '--save', f'../checkpoints/{model}',
    '--batch-size', '128'
]
args = parse_arguments()


pretrained_checkpoint = f'../checkpoints/{model}/zeroshot.pt'
finetuned_checkpoints = [f'../checkpoints/{model}/{dataset}/finetuned.pt' for dataset in datasets]

In [10]:
operations = ['subtract', 'add', 'multiply', 'divide']


In [ ]:
results = {}
for dataset in datasets:
    print(f"Testing dataset: {dataset}")

    finetuned_checkpoint = f'../checkpoints/{model}/{dataset}/finetuned.pt'
    for operation in operations:
        task_vector = TaskVectorAdapted(
            pretrained_checkpoint=pretrained_checkpoint,
            finetuned_checkpoint=finetuned_checkpoint,
            operation=operation
        )
        print(f"Testing operation: {operation}")
        for i in range(10):
            print(f"Iteration {i+1}/10")
            encoder_test = task_vector.apply_to(pretrained_checkpoint, scaling_coef=0.1*i)
            metrics = eval_single_dataset(encoder_test, dataset, args)
            results[(dataset, operation, i)] = metrics
            del encoder_test
        del task_vector


Testing dataset: Cars
Testing operation: subtract
Iteration 1/10
Classification head for ../ViT-L-14 on Cars exists at ../checkpoints/ViT-L-14/head_Cars.pt
Loading classification head from ../checkpoints/ViT-L-14/head_Cars.pt


/Users/giovanniattina/miniconda3/envs/task-vectors/lib/python3.10/site-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
  0%|          | 0/63 [01:20<?, ?it/s]


Done evaluating on Cars. Accuracy: 75.78%
Iteration 2/10
Classification head for ../ViT-L-14 on Cars exists at ../checkpoints/ViT-L-14/head_Cars.pt
Loading classification head from ../checkpoints/ViT-L-14/head_Cars.pt


  0%|          | 0/63 [01:24<?, ?it/s]


Done evaluating on Cars. Accuracy: 81.25%
Iteration 3/10
Classification head for ../ViT-L-14 on Cars exists at ../checkpoints/ViT-L-14/head_Cars.pt
Loading classification head from ../checkpoints/ViT-L-14/head_Cars.pt


...

  0%|          | 0/79 [00:46<?, ?it/s]


Done evaluating on MNIST. Accuracy: 10.16%
Iteration 6/10
Classification head for ../ViT-L-14 on MNIST exists at ../checkpoints/ViT-L-14/head_MNIST.pt
Loading classification head from ../checkpoints/ViT-L-14/head_MNIST.pt


  0%|          | 0/79 [00:45<?, ?it/s]


Done evaluating on MNIST. Accuracy: 9.38%
Iteration 7/10
Classification head for ../ViT-L-14 on MNIST exists at ../checkpoints/ViT-L-14/head_MNIST.pt
Loading classification head from ../checkpoints/ViT-L-14/head_MNIST.pt


  0%|          | 0/79 [00:00<?, ?it/s]

In [1]:
results

NameError: name 'results' is not defined

In [21]:
import pandas as pd

In [22]:
pd.DataFrame.from_dict(results, orient='index').reset_index().rename(columns={'index': 'dataset_operation'})

,level_0,level_1,top1
0,Cars,add,0.000000
1,Cars,divide,0.007812
2,Cars,multiply,0.015625
3,Cars,subtract,0.914062
4,MNIST,add,0.125000
5,MNIST,divide,0.078125
6,MNIST,multiply,0.093750
7,MNIST,subtract,1.000000
